In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio pympi-ling scikit-learn

In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset, Dataset
import os
from pydub import AudioSegment
import pympi
import pandas as pd
from sklearn.model_selection import train_test_split

model_id = "/home/ziane212/projects/MMS_ASR_finetuning/whisper-medium-CAENNAIS"

/home/ziane212/anaconda3/envs/finetuning_whisper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def build_dataset_from_eaf(eaf_files, wav_files, output_dir, name_dataset):
    # Vérifiez que le nombre de fichiers EAF et WAV correspond
    if len(eaf_files) != len(wav_files):
        raise ValueError("Le nombre de fichiers EAF et WAV doit être le même.")
    
    # Préparer une liste pour stocker les données du tableau TSV
    data = []
    
    # Initialiser un compteur pour les annotations passées
    annotations_passed = 0
    
    # Parcourir chaque paire de fichiers EAF et WAV
    for eaf_file, wav_file in zip(eaf_files, wav_files):
        # Extraire le nom du fichier sans extension pour l'utiliser dans les noms des extraits
        file_base_name = os.path.basename(eaf_file).replace('.eaf', '')
        
        print(f"Traitement de {file_base_name}")
        
        # Charger le fichier EAF
        eaf = pympi.Elan.Eaf(eaf_file)
        
        # Charger le fichier audio
        audio = AudioSegment.from_wav(wav_file)
        
        # Parcourir les tiers et les annotations
        for tier_name, tier in eaf.tiers.items():
            if "_extra" not in tier_name:
                for annotation in eaf.get_annotation_data_for_tier(tier_name):
                    start_time, end_time, value = annotation
                    duration = (end_time - start_time) / 1000  # Convertir en secondes
                    value = value.strip()  # Supprimer les espaces superflus
                    
                    # Vérifier les conditions de filtrage
                    if duration > 30 or (value.startswith("[") and value.endswith("]")):
                        annotations_passed += 1
                        continue
                    
                    # Découper l'extrait audio selon les timecodes
                    extract = audio[start_time:end_time]
                    
                    # Définir le chemin de sortie pour l'extrait, incluant le nom du fichier source
                    output_file = os.path.join(output_dir, f"{file_base_name}_{tier_name}_{start_time}_{end_time}.wav")
                    os.makedirs(os.path.dirname(output_file), exist_ok=True)
                    
                    # Sauvegarder l'extrait
                    extract.export(output_file, format="wav")
                    
                    # Ajouter les informations dans la liste des données
                    timecodes = f"[{start_time}, {end_time}]"  # Format des timecodes
                    data.append([output_file, value, timecodes, tier_name])
    
    # Créer un DataFrame pandas pour le tableau TSV avec les colonnes supplémentaires
    df = pd.DataFrame(data, columns=["audio", "text", "timecodes", "speaker"])
    
    # Définir un sous-dossier pour le TSV
    tsv_dir = os.path.join(output_dir, "tsv_files")
    os.makedirs(tsv_dir, exist_ok=True)
    
    # Chemin pour le fichier TSV
    tsv_file = os.path.join(tsv_dir, name_dataset)
    df.to_csv(tsv_file, sep='\t', index=False)
    
    print(f"Process completed. TSV file saved in {tsv_dir}.")
    print(f"Number of annotations passed: {annotations_passed}")

    return tsv_file

def load_model_dataset(path_in, model_id): 
    file_dict = {
    "to_pred" : path_in 
    }

    d_audio = load_dataset('csv',
                        data_files=file_dict,
                        delimiter='\t',
                        column_names=['audio', 'text', 'timecodes', 'speaker'],
                        skiprows=1
                        )

    d_audio = d_audio['to_pred'].cast_column("audio", Audio(sampling_rate=16_000))

    # load model and processor
    try:
        processor = WhisperProcessor.from_pretrained(model_id)
        model = WhisperForConditionalGeneration.from_pretrained(model_id)
        model = model.to("cuda")  # Déplacer le modèle sur le GPU
        print(f"Model {model_id} is loaded.")
    except:
        print(f"Error: Fail to load model {model_id}.")

    return processor, model, d_audio

def transcribe(processor, model, d_audio):
    # Liste pour stocker les nouvelles données
    updated_data = []

    for i in d_audio:
        # Charger les données d'entrée
        input_dict = processor(
            i["audio"]["array"],
            sampling_rate=16_000,
            return_tensors="pt",
            padding=True
        ).input_features.to("cuda")  # Déplacer l'entrée sur le GPU

        # Effectuer la prédiction
        
        predicted_ids = model.generate(input_dict)  # Pas besoin de transférer à nouveau ici

        # Décoder la sortie
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        # Ajouter les informations mises à jour
        updated_data.append({
            "audio": i["audio"]["path"],  # Conserver uniquement le chemin du fichier
            "text": transcription,  # Remplacer le texte
            "timecodes": i["timecodes"],
            "speaker": i["speaker"]
        })

    # Créer un nouveau dataset avec les données mises à jour
    new_dataset = Dataset.from_list(updated_data)
    return new_dataset

In [3]:
eaf_files = [
    '/home/ziane212/projects/transcription/data/CAENNAIS/GB/E2_GB_WAV_IL.eaf',
]

wav_files = [
    '/home/ziane212/projects/transcription/data/CAENNAIS/GB/E2_GB_WAV.wav',
]

output_dir = '/home/ziane212/projects/MMS_ASR_finetuning/caennais_E2GB_20.11.24/audio'
name_dataset = "dataset_caennais_E2GB_20.11.24.tsv"

def process_file(eaf_files, 
                 wav_files,
                 output_dir,
                 name_dataset,
                 model_id):
    
    path_in = build_dataset_from_eaf(eaf_files,
                                     wav_files,
                                     output_dir,
                                     name_dataset)
    
    processor, model, d_audio = load_model_dataset(path_in, model_id)
    
    d_audio = transcribe(processor, model, d_audio)

    return d_audio

In [4]:
d_audio = process_file(eaf_files, 
                 wav_files,
                 output_dir,
                 name_dataset,
                 model_id)

Traitement de E2_GB_WAV_IL
Process completed. TSV file saved in /home/ziane212/projects/MMS_ASR_finetuning/caennais_E2GB_20.11.24/audio/tsv_files.
Number of annotations passed: 0


Generating to_pred split: 1712 examples [00:00, 177330.61 examples/s]


Model /home/ziane212/projects/MMS_ASR_finetuning/whisper-medium-CAENNAIS is loaded.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
d_audio[20]

{'audio': '/home/ziane212/projects/MMS_ASR_finetuning/caennais_E2GB_20.11.24/audio/E2_GB_WAV_IL_LOCB2_455360_458400.wav',
 'text': "est-ce que c'est normal d' l’ avoir euh quand on",
 'timecodes': '[455360, 458400]',
 'speaker': 'LOCB2'}

In [6]:
output_path = "/home/ziane212/projects/MMS_ASR_finetuning/caennais_E2GB_20.11.24/audio/dataset_caennais_E2GB_out_20.11.24.tsv"
d_audio.to_csv(output_path, sep='\t', index=False)

Creating CSV from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 214.18ba/s]


281578

In [7]:
import pympi
import pandas as pd

def update_eaf_with_tsv(eaf_file, tsv_file, output_eaf):
    """
    Met à jour un fichier EAF en utilisant les informations d'un fichier TSV.
    
    Args:
        eaf_file (str): Chemin du fichier EAF d'entrée.
        tsv_file (str): Chemin du fichier TSV contenant les informations.
        output_eaf (str): Chemin du fichier EAF de sortie modifié.
    """
    # Charger le fichier EAF
    eaf = pympi.Elan.Eaf(eaf_file)
    
    # Charger le fichier TSV
    df = pd.read_csv(tsv_file, sep='\t')

    # Parcourir chaque ligne du fichier TSV
    for _, row in df.iterrows():
        speaker = row['speaker']
        text = row['text']
        timecodes = eval(row['timecodes'])  # Convertir la chaîne en liste
        
        start_time, end_time = timecodes
        
        # Vérifier si le tier existe
        if speaker in eaf.tiers:
            # Parcourir les annotations dans le tier
            for i, annotation in enumerate(eaf.get_annotation_data_for_tier(speaker)):
                ann_start, ann_end, ann_value = annotation
                if ann_value == "":
                    # Vérifier si les timecodes correspondent
                    if int(start_time) == ann_start and int(end_time) == ann_end:
                        # Mettre à jour le texte de l'annotation
                        eaf.remove_annotation(speaker, ann_start, ann_end)
                        eaf.add_annotation(speaker, ann_start, ann_end, value=text)
                        print(f"Annotation mise à jour : {speaker}, {start_time}-{end_time}, {text}")
    
    # Sauvegarder le fichier EAF modifié
    eaf.to_file(output_eaf)
    print(f"Fichier EAF modifié sauvegardé sous : {output_eaf}")

# Exemple d'utilisation
eaf_file = "/home/ziane212/projects/transcription/data/CAENNAIS/GB/E2_GB_WAV_IL.eaf"
tsv_file = "/home/ziane212/projects/MMS_ASR_finetuning/caennais_E2GB_20.11.24/audio/dataset_caennais_E2GB_out_20.11.24.tsv"
output_eaf = "/home/ziane212/projects/transcription/data/CAENNAIS/GB/E2_GB_transAuto.eaf"

update_eaf_with_tsv(eaf_file, tsv_file, output_eaf)


Annotation mise à jour : LOCB2, 1562515-1562882, mmh
Annotation mise à jour : LOCB2, 1576898-1577388, ouais
Annotation mise à jour : LOCB2, 1581276-1581586, ouais
Annotation mise à jour : LOCB2, 1582155-1585310, et puis qui a ce respect envers la
Annotation mise à jour : LOCB2, 1585690-1586760, oui les profs aussi
Annotation mise à jour : LOCB2, 1587070-1588707, donc ça dans ma tête oui c'est
Annotation mise à jour : LOCB2, 1589293-1590259, les élèves ils éc-
Annotation mise à jour : LOCB2, 1591534-1592310, je sais pas
Annotation mise à jour : LOCB2, 1592776-1594621, c'est peut-être pas ce que je confonds
Annotation mise à jour : LOCB2, 1595052-1598672, le respect ou le res- quand je dis le respect c'est que en fait on
Annotation mise à jour : LOCB2, 1598983-1603293, y’ a une plus grande distance et puis que qu’on dit madame et puis qu’on voit
Annotation mise à jour : LOCB2, 1603603-1605224, en fait c’est pour moi ça c'est
Annotation mise à jour : LOCB2, 1605603-1607586, c'est pour ça 